In [ ]:
pip install pytesseract


In [ ]:
import cv2
import pytesseract

# Initialize the video capture object
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the video capture object
    ret, frame = cap.read()
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to segment the foreground from the background
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Find contours in the image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate through each contour and filter for number plates and helmets
    for cnt in contours:
        # Compute the bounding box of the contour
        x, y, w, h = cv2.boundingRect(cnt)

        # Filter for number plates
        if w/h > 2 and w/h < 5 and w > 100 and h > 30:
            # Crop the number plate region
            plate_roi = frame[y:y+h, x:x+w]

            # Apply OCR using pytesseract library to extract the number plate text
            plate_text = pytesseract.image_to_string(plate_roi, lang='eng', config='--psm 7')

            # Print the number plate text
            print("Number plate detected:", plate_text)

        # Filter for helmets
        if w/h > 0.4 and w/h < 0.6 and w > 50 and h > 50:
            # Draw a rectangle around the helmet
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

    # Display the frame with detected number plates and helmets
    cv2.imshow("Detected plates and helmets", frame)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()
